In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from eodhd import APIClient
from sklearn.preprocessing import StandardScaler

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

In [ ]:
api = APIClient("6608304b3de2b5.23968015")

In [ ]:
resp = api.get_eod_historical_stock_market_data(
    symbol="AAPL.US",
    period="d",
    order="a",
)
df = pd.DataFrame(resp)
df.set_index(df.columns[0], inplace=True)
df.index = pd.to_datetime(df.index)
data = df[["adjusted_close", "volume"]]
data = data.astype(np.float32)

In [ ]:
data.shape

(11052, 2)

In [ ]:
train, test = (
    data.loc[data[data.index < "2021-01-01"].index],
    data.loc[data[data.index >= "2021-01-01"].index],
)
train.shape, test.shape

((10100, 2), (952, 2))

In [ ]:
train.head()

,adjusted_close,volume
date,,
1980-12-12,0.09890000,469033600.00000000
1980-12-15,0.09380000,175884800.00000000
1980-12-16,0.08690000,105728000.00000000
1980-12-17,0.08910000,86441600.00000000
1980-12-18,0.09170000,73449600.00000000


In [ ]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

In [ ]:
def create_windows(data, win_size):
    X = []
    for i in range(len(data) - win_size):
        X.append(data[i : i + win_size])
    return np.array(X), np.array(X)[:, :, 0]


WIN_SIZE = 30
train_windows, train_labels = create_windows(train_scaled, WIN_SIZE)
test_windows, test_labels = create_windows(test_scaled, WIN_SIZE)

In [ ]:
train_windows.shape, train_labels.shape, test_windows.shape, test_labels.shape

((10070, 30, 2), (10070, 30), (922, 30, 2), (922, 30))

# Model definition and training

In [ ]:
from keras import layers
from keras.models import Sequential

## AE

In [ ]:
ae1 = Sequential(name="autoencoder")
ae1.add(layers.InputLayer(shape=(30, 2)))
ae1.add(layers.LSTM(32))
ae1.add(layers.Dropout(rate=0.2))
ae1.add(layers.RepeatVector(30))
ae1.add(layers.LSTM(16, return_sequences=True))
ae1.add(layers.Dropout(rate=0.2))
ae1.add(layers.TimeDistributed(layers.Dense(1)))
ae1.add(layers.Reshape((30,)))
ae1.compile(optimizer="adam", loss="mae")
ae1.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 16)         │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 1)          │            17 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 30)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,321 (9.07 KB)

 Trainable params: 2,321 (9.07 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
ae = Sequential(name="autoencoder")
ae.add(layers.InputLayer(shape=(train_windows.shape[1], train_windows.shape[2])))
ae.add(layers.LSTM(32))
ae.add(layers.Dropout(rate=0.2))
ae.add(layers.RepeatVector(train_windows.shape[1]))
ae.add(layers.LSTM(16, return_sequences=True))
ae.add(layers.Dropout(rate=0.2))
ae.add(layers.TimeDistributed(layers.Dense(1)))
ae.add(layers.Reshape((train_windows.shape[1],)))
ae.compile(optimizer="adam", loss="mae")
ae.summary()

# ae1 = Sequential(name="autoencoder")
# ae1.add(layers.InputLayer(shape=(30, 2)))
# ae1.add(layers.LSTM(32))
# ae1.add(layers.Dropout(rate=0.2))
# ae1.add(layers.RepeatVector(30))
# ae1.add(layers.LSTM(16, return_sequences=True))
# ae1.add(layers.Dropout(rate=0.2))
# ae1.add(layers.TimeDistributed(layers.Dense(1)))
# ae1.add(layers.Reshape((30,)))
# ae1.compile(optimizer="adam", loss="mae")
# ae1.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         4,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 16)         │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 1)          │            17 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 30)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,633 (29.82 KB)

 Trainable params: 7,633 (29.82 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = ae.fit(
    train_windows,
    train_labels,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    shuffle=False,
)

Epoch 1/30


KeyboardInterrupt: 

In [ ]:
train_pred = ae.predict(train_windows)
train_mae_loss = np.mean(np.abs(train_pred - train_labels), axis=1)


threshold = np.percentile(train_mae_loss, 75)
threshold

px.histogram(train_mae_loss[train_mae_loss < threshold], title="Train MAE Loss")

315/315 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step


In [ ]:
test_pred = ae.predict(test_windows)
test_mae_loss = np.mean(np.abs(test_pred - test_labels), axis=1)

max(test_mae_loss)
px.histogram(test_mae_loss, title="Test MAE Loss")

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [ ]:
test_mae_loss.shape, test_scaled.shape

((913,), (943, 2))

In [ ]:
is_anomaly = test_mae_loss > 7
anomalies = test[:-WIN_SIZE][is_anomaly]
anomalies.shape

(492, 2)

In [ ]:
fig = px.line(
    test,
    x=test.index,
    y="adjusted_close",
    title="AAPL Stock Price",
    width=1200,
)

fig.add_trace(
    go.Scatter(
        x=anomalies.index,
        y=test.loc[anomalies.index, "adjusted_close"],
        mode="markers",
        marker=dict(color="red", size=3),
        name="Anomaly",
    )
)

fig.show()

In [ ]:
import os

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from eodhd import APIClient
from IPython.display import display
from keras import layers
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler

In [ ]:
os.environ["KERAS_BACKEND"] = "tensorflow"
api = APIClient("6608304b3de2b5.23968015")

In [ ]:
def on_button_click(b):
    with output:
        output.clear_output()

        resp = api.get_eod_historical_stock_market_data(
            symbol=dropdown.value,
            period="d",
            order="a",
        )

        df = pd.DataFrame(resp)
        df.set_index(df.columns[0], inplace=True)
        df.index = pd.to_datetime(df.index)
        data = df[["adjusted_close", "volume"]]
        data = data.astype(np.float32)

        plt.figure(figsize=(10, 6))
        plt.plot(data.index, data["adjusted_close"], label="Adjusted Close Price")
        plt.title(f"{dropdown.value} Stock Price")
        plt.xlabel("Date")
        plt.ylabel("Adjusted Close Price")
        plt.grid(True)
        plt.legend()

        plt.show()

In [ ]:
dropdown = widgets.Dropdown(
    options=["AAPL.US", "MSFT.US", "GOOGL.US"],
    value="AAPL.US",
    description="Pick:",
    disabled=False,
)

button = widgets.Button(
    description="Submit",
    disabled=False,
    button_style="",
    tooltip="Click me",
)

output = widgets.Output()

button.on_click(on_button_click)

hbox = widgets.HBox([dropdown, button])

display(hbox)
display(output)


Output()